<a href="https://colab.research.google.com/github/dhanushpachabhatla/deep-learning-notebook/blob/main/keras_hyperparamter_kuning_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Keras-tuner library implementation

In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('/content/diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [7]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
X.shape

(768, 8)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [47]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [12]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim = 8))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
history = model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.4558 - loss: 0.7722 - val_accuracy: 0.5065 - val_loss: 0.7116
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5968 - loss: 0.6788 - val_accuracy: 0.5779 - val_loss: 0.6584
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6580 - loss: 0.6312 - val_accuracy: 0.6364 - val_loss: 0.6193
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6511 - loss: 0.6121 - val_accuracy: 0.6623 - val_loss: 0.5885
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6712 - loss: 0.5756 - val_accuracy: 0.7143 - val_loss: 0.5636
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7252 - loss: 0.5347 - val_accuracy: 0.7338 - val_loss: 0.5447
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7382 - loss: 0.5252 - val_accuracy: 0.7597 - val_loss: 0.5294
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7191 - loss: 0.5393 - val_accuracy: 0.7662 - 

Todo:
1. how to select appropriate optimizer
2. No. of nodes in a layer
3. No. of select no. of layers
4. All in one model

In [14]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.9 MB/s eta 0:00:00


In [16]:
import kerastuner as kt

<ipython-input-16-1654478174>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


#how to select best optimizer?

In [18]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim = 8))
  model.add(Dense(1,activation = 'sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [20]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials = 5)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
tuner.search(X_train,y_train,epochs = 5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 04s]
val_accuracy: 0.6883116960525513

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 13s


In [23]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [25]:
model = tuner.get_best_models(num_models=1)[0]

In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
history_2 = model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7019 - loss: 0.5986 - val_accuracy: 0.7662 - val_loss: 0.5522
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7663 - loss: 0.5349 - val_accuracy: 0.7662 - val_loss: 0.5307
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7526 - loss: 0.5363 - val_accuracy: 0.7597 - val_loss: 0.5136
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7375 - loss: 0.5253 - val_accuracy: 0.7532 - val_loss: 0.5013
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7608 - loss: 0.5033 - val_accuracy: 0.7532 - val_loss: 0.4923
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7622 - loss: 0.5002 - val_accuracy: 0.7597 - val_loss: 0.4833
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7782 - loss: 0.4912 - val_accuracy: 0.7662 - val_loss: 0.4787
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7532 - loss: 0.4945 - val_accuracy: 0.77

#how to select no. of neurons?

In [29]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units', min_value = 8,max_value = 128,step = 8)
  model.add(Dense(units=units,activation='relu',input_dim = 8))
  model.add(Dense(1,activation = 'sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [32]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials = 5,
                        directory = 'mydir',
                        project_name = 'helloworld')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
tuner.search(X_train,y_train,epochs = 5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 15s


In [34]:
tuner.get_best_hyperparameters()[0].values

{'units': 56}

In [35]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [36]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7549 - loss: 0.4968 - val_accuracy: 0.7922 - val_loss: 0.4814
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7523 - loss: 0.4873 - val_accuracy: 0.7857 - val_loss: 0.4756
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7834 - loss: 0.4670 - val_accuracy: 0.7792 - val_loss: 0.4721
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7869 - loss: 0.4566 - val_accuracy: 0.7857 - val_loss: 0.4683
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7918 - loss: 0.4425 - val_accuracy: 0.7922 - val_loss: 0.4660
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8079 - loss: 0.4595 - val_accuracy: 0.7857 - val_loss: 0.4642
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7768 - loss: 0.4585 - val_accuracy: 0.7857 - val_loss: 0.4619
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7808 - loss: 0.4473 - val_accuracy: 0.7

#how to select no. of layers?

In [37]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(units=56,activation='relu',input_dim = 8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(units=56,activation='relu'))
  model.add(Dense(1,activation = 'sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [38]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials = 5,
                        directory = 'mydir',
                        project_name = 'num_layers')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
tuner.search(X_train,y_train,epochs = 5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 23s


In [40]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [41]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

#All in one

In [55]:
def build_model(hp):
    model = Sequential()

    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        units = hp.Int(f'units_{i}', min_value=8, max_value=128, step=8)
        activation = hp.Choice(f'activation_{i}', values=['relu', 'tanh', 'sigmoid'])
        dropout_rate = hp.Choice(f'dropout_{i}', values=[0.1 * j for j in range(1, 10)])  # 0.1 to 0.9

        if i == 0:
            model.add(Dense(units=units, activation=activation, input_dim=8))
        else:
            model.add(Dense(units=units, activation=activation))

        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid'))  # Output layer

    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [56]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials = 5,
                        directory = 'mydir',
                        project_name = 'all_in_one_2')

In [57]:
tuner.search(X_train,y_train,epochs = 5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 25s


In [58]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units_0': 48,
 'activation_0': 'tanh',
 'dropout_0': 0.5,
 'optimizer': 'sgd',
 'units_1': 64,
 'activation_1': 'relu',
 'dropout_1': 0.7000000000000001,
 'units_2': 32,
 'activation_2': 'relu',
 'dropout_2': 0.4,
 'units_3': 8,
 'activation_3': 'relu',
 'dropout_3': 0.1,
 'units_4': 8,
 'activation_4': 'relu',
 'dropout_4': 0.1,
 'units_5': 8,
 'activation_5': 'relu',
 'dropout_5': 0.1,
 'units_6': 8,
 'activation_6': 'relu',
 'dropout_6': 0.1}

In [59]:
model = tuner.get_best_models(num_models = 1)[0]

In [60]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.6582 - loss: 0.6770 - val_accuracy: 0.6429 - val_loss: 0.6815
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6852 - loss: 0.6550 - val_accuracy: 0.6429 - val_loss: 0.6783
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6434 - loss: 0.6762 - val_accuracy: 0.6429 - val_loss: 0.6758
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6637 - loss: 0.6532 - val_accuracy: 0.6429 - val_loss: 0.6737
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6471 - loss: 0.6727 - val_accuracy: 0.6429 - val_loss: 0.6716
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6540 - loss: 0.6656 - val_accuracy: 0.6429 - val_loss: 0.6693
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6492 - loss: 0.6664 - val_accuracy: 0.6429 - val_loss: 0.6678
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6397 - loss: 0.6743 - val_accuracy: 0.64